# Toward Skynet: Intro to AI

###Overview
There are **many** machine learning algorithms. These can be broken down based on what they try to do, and how they are trained.
* Supervision
  - Supervised: learns to classify data based on provided examples
  - Unsupervised: learns to cluster data without human supervision
* Training
  - Online: learns by looking at each data point once (like in data mining)
  - Active: learns by looking at batches of data multiple times
* Models 
  - Simple learning: assumes that the correct model can be found. This is a good assumption but is often impossible
  - Agnostic learning: accepts that using a certain algorithm, the perfect model can't always be created.

###Analyzing Algorithms
* **Representation**: how well can this problem be represented with this model?
* **Optimization**: how much can we reduce the error with more training?
* **Generalization**: how well does our model perform on unseen testing data?
* Problems: does our model underfit? overfit?
  - Underfitting: our model isn't expressive enough to represent how the data interact. Solve this by adding more layers, adding more features, etc.
  - Overfitting: our model is **too** expressive. This is a more common problem and is an area of active research. How do we solve this? Nobody knows, but regularization, dropout, and limiting features are good starts.
  
###Evaluation
* How do we see how good our machine learning model is?
* We cannot test on training data! Can you see why?
* We save a portion of our labeled data as a test set and don't train using it.
* **Cross-validation** is where we split up the labeled data into many sets (often 5 or 10) and train using all but one of them, and test using the last one. This is often more accurate. 

###Decision Trees

####Motivation
Given a set of examples, decide how to label the examples into classes. Decision trees try to model human decision processes by looking at one feature at a time. This forms a tree (or flowchart) based on which features improve accuracy the most

####Measuring Accuracy
We measure how good a decision tree is by looking at its **entropy**, or how noisy the nodes are. This is a weighted sum of how good each node is at classifying examples, multiplied by the relative sizes of the nodes. So, the performance of larger nodes is more important than the performance of smaller ones.
<a href="https://www.codecogs.com/eqnedit.php?latex=\sum_{leaf}&space;\left(\frac{|leaf|}{|Z|}&space;\sum_{v}&space;-Pr(v)*log(Pr(v))&space;\right&space;)" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\sum_{leaf}&space;\left(\frac{|leaf|}{|Z|}&space;\sum_{v}&space;-Pr(v)*log(Pr(v))&space;\right&space;)" title="\sum_{leaf} \left(\frac{|leaf|}{|Z|} \sum_{v} -Pr(v)*log(Pr(v)) \right )" /></a>

In [ ]:
from math import log
def entropy(arr):
    """Computes the entropy of an array of example tuples"""
    if all(isinstance(elmnt, tuple) for elmnt in arr):
        labels = {}
        for elmnt in arr:
            if elmnt[-1] not in labels:
                labels[elmnt[-1]] = 1
            else:
                labels[elmnt[-1]] += 1
        for label in labels:
            labels[label] /= float(len(arr))
        return sum([-1*labels[index]*log(labels[index],2) for index in labels])
    else:
        raise ValueError("Attempting to calculate entropy of a non-leaf node")

####Decision Tree Algorithm
The decision tree starts out as a single node with all of the examples. Each example has features `(f1, f2, f3, ..., fn)` and a label `v`. The decision tree algorithm is fairly simple compared to many others.
1. Consider each feature `f`. Calculate the entropy `H(s)` for each split according to the equation above.
2. Split the tree on this feature `f`. All of the examples are moved to their corresponding leaves.
3. If each node is homogeneous (labelled the same), stop the algorithm.
4. If there are no more features to split on, stop the algorithm.
5. For each non-homogeneous node, repeat from step 1. on this new leaf.

In [2]:
def selectAttribute(arr, verbose=False):
    """If arr is not homogeneous, returns index of attribute with greatest information gain"""
    current_entropy = entropy(arr)
    entropy_arr = []
    for index in xrange(len(arr[0])-1):
        sum_entropy = sum([entropy(leaf)*len(leaf)/len(arr) for leaf in split(arr, index).values()])
        entropy_arr.append((index, sum_entropy))
        if verbose: print("Information gain for attribute "+str(index)+": "+str(current_entropy-sum_entropy))
    best_att = min(entropy_arr, key=lambda x:x[1])
    return best_att[0] if best_att[1] < current_entropy else None  # check to see if no information remains

In [3]:
def decision_tree(tree, verbose=False):
    """Recursively perform splits to form decision tree"""
    selected_attribute = selectAttribute(tree, verbose=verbose)
    if isHomogeneous(tree) or selected_attribute is None:  # if should not be split any further
        labels = {}
        for elmnt in tree:
            if elmnt[-1] not in labels:
                labels[elmnt[-1]] = 1
            else:
                labels[elmnt[-1]] += 1
        label = max([(label,labels[label]) for label in labels], key=lambda x:x[1])[0]
        if verbose: print "\033[0mCurrent node is: "+str(tree)+"\n\033[94mNot splitting, label: "+label+"\033[0m\n"
        return label
    else:
        best_att = selected_attribute
        if verbose: print "\033[0mCurrent node is: "+str(tree)+"\n\033[92mSplitting on: \033[0m"+str(best_att)+"\n"
        node = {}
        children = split(tree, best_att)
        for child in children:
            node[child] = decision_tree(children[child], verbose)
        return (best_att, node)

In [4]:
def classify(event, tree):
    if not isinstance(tree, tuple):
        print "Example "+str(event)+" has been labeled as "+tree
        return tree # reached a label
    else:
        return classify(event, tree[1][event[tree[0]]])

####Other ML algorithms
* Naive Bayes: This is often considered to be the easiest and is often the first ML algorithm that people learn. It is prone to **underfitting** and is not very expressive, but performs surprisingly well
<a href="https://www.codecogs.com/eqnedit.php?latex=Bayes'&space;Theorem:&space;Pr(x|y)&space;=&space;\frac{Pr(y|x)*Pr(x)}{Pr(y)}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?Bayes'&space;Theorem:&space;Pr(x|y)&space;=&space;\frac{Pr(y|x)*Pr(x)}{Pr(y)}" title="Bayes' Theorem: Pr(x|y) = \frac{Pr(y|x)*Pr(x)}{Pr(y)}" /></a>
* Reinforcement Learning: This is useful for long processes without easily labeled state values, like chess or Atari games. The agent learns by playing or observing for a long period of time and picking up rewards and punishments.
* Neural Nets: This is the state of the art but is fairly computationally intensive. They try to model neurons.
  - Deep learning: multi-layer neural networks
  - Convolutional Neural Networks: good for processing images
  - Recurrent Neural Networks: deal with sequences of data
  - Generative Adversarial Networks: generate data from labels, eg. generating mugshots from profiles
